In [1]:
import os
import sys
sys.path.insert(0,'../')
from libs.ssl_data import *
from libs.ssl_model import *
from libs.ssl_utils import *
from libs.ssl_task import *
from sklearn.model_selection import train_test_split
# Use these commands in the same cell to autoreload local library source code
# https://stackoverflow.com/questions/45807760/change-source-code-and-update-jupyter-notebook-but-retain-variables
%load_ext autoreload
%autoreload 2

In [1]:
def preprocess_fn(ds, savedir, hp):
    from sklearn.preprocessing import scale as standard_scale
    os.makedirs(savedir, exist_ok=True)

    def global_norm(data, method='standard'):
        if method == 'standard':
            center = np.mean(data)  
            variance = np.std(data)  
        elif method == 'robust':
            center = np.median(data)  
            variance = np.percentile(data, 75) - np.percentile(data, 25)
        # standard scale to 0 mean and 1 std using statistics of the entire recording
        data = (data - center) / variance # normalize preserving batch dim
        return data

    sampling_rate = 250 # resample to follow the tutorial sampling rate
    # Factor to convert from uV to V
    factor = 1e6
    channels = ds.datasets[0].raw.info['ch_names']
    preprocessors = [
        Preprocessor('set_channel_types', mapping=dict(zip(channels, ['eeg']*len(channels)))),
        Preprocessor('notch_filter', freqs=(60, 120)),    
        Preprocessor('filter', l_freq=hp, h_freq=59),
        Preprocessor('resample', sfreq=sampling_rate),
        Preprocessor('set_eeg_reference', ref_channels=['Cz']),
        # Preprocessor('crop', tmin=10),  # crop first 10 seconds as begining of noise recording
        Preprocessor('drop_channels', ch_names=['Cz']),  # discard Cz
        Preprocessor(global_norm, method='robust', channel_wise=True),  
        # Preprocessor(standard_scale, channel_wise=True), # normalization for deep learning
    ]
    # Transform the data
    preprocess(ds, preprocessors, save_dir=savedir, overwrite=True, n_jobs=10, copy_data=True)

    return ds

In [ ]:
datasets = ['ds005505', 'ds005506', 'ds005507','ds005508','ds005509','ds005510', 'ds005511', 'ds005512', 'ds005514', 'ds005515', 'ds005516']
for hp in [0.1]: #[0.01, 0.5, 1.0]:
    for dsnumber in datasets:
        folder_name = f'data-hp_{hp}-robust_recording_channelwise'
        # os.makedirs(Path('/mnt/nemar/dtyoung/eeg-ssl-data') / folder_name, exist_ok=True)
        os.makedirs(folder_name, exist_ok=True)
        ds = EEGDashDataset({'dataset': dsnumber, 'task': 'RestingState'}, cache_dir='/mnt/nemar/openneuro', description_fields=['subject', 'session', 'run', 'task', 'age', 'gender', 'sex', 'p_factor', 'externalizing', 'internalizing', 'attention'])
        # savedir = Path('/mnt/nemar/dtyoung/eeg-ssl-data') / folder_name / f'{dsnumber}_preprocessed'
        savedir = Path(folder_name) / f'{dsnumber}_preprocessed'
        try:
            ds = preprocess_fn(ds, savedir, hp=hp)
        except Exception as e:
            # append the error to a file
            with open(f'error_log_{folder_name}.txt', 'a') as f:
                f.write(f"Error processing {dsnumber}: {str(e)}\n")


/home/dung/Documents/EEG-Dash-Data/eegdash/data_utils.py:96: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  self._raw = mne_bids.read_raw_bids(self.bidspath, verbose=False)
/home/dung/Documents/EEG-Dash-Data/eegdash/data_utils.py:96: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  self._raw = mne_bids.read_raw_bids(self.bidspath, verbose=False)
/home/dung/Documents/EEG-Dash-Data/eegdash/data_utils.py:96: RuntimeWarning: Unable to map the following column(s) to to MNE:
release_number: R1
ehq_total: 57.83
commercial_use: Yes
full_pheno: Yes
p_factor: -0.648
attention: -0.055
internalizing: -0.051
externalizing: -0.109
RestingState: available
DespicableMe: available
FunwithFractals: available
ThePresent: available
DiaryOfAWimpyKid: available
contrastChangeDetection_1: unavailable
contrastChangeDetection_2:

Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3301 samples (6.602 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.2s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 59 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 59.00 Hz
- Upper transition bandwidth: 14.75 Hz (-6 dB cutoff frequency: 66.38 Hz)
- Filter length: 16501 samples (33.002 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.2s


...

Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3301 samples (6.602 s)



[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 59 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 59.00 Hz
- Upper transition bandwidth: 14.75 Hz (-6 dB cutoff frequency: 66.38 Hz)
- Filter length: 16501 samples (33.002 s)



[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.2s


EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Writing /mnt/nemar/dtyoung/eeg-ssl-data/data-hp_0.1-robust_recording_channelwise/ds005508_preprocessed/0/0-raw.fif
Closing /mnt/nemar/dtyoung/eeg-ssl-data/data-hp_0.1-robust_recording_channelwise/ds005508_preprocessed/0/0-raw.fif
[done]
